### key.py

In [1]:
from jproperties import Properties
key="webvitals_config.properties"
try:
    f = open(key)
    configs = Properties()
    
    with open(key, 'rb') as config_file:
        configs.load(config_file)
        
    items_view = configs.items()
    list_keys = []
    print("'" + key + "'" + " found!")    
    for item in items_view:
        list_keys.append(item[0])    
except IOError:
    print("please create " + key + " file w/ credentials")
finally:
    f.close()

'webvitals_config.properties' found!


### automate_login.py

In [2]:
def login(username, password):
    
    from logging import error
    from selenium import webdriver
    from selenium.webdriver.support.ui import WebDriverWait
    
    # Login
    driver.find_element_by_name("ssousername").send_keys(username)
    driver.find_element_by_name("password").send_keys(password)
    driver.find_element_by_xpath("/html/body/center/table/tbody/tr[3]/td/font/input[1]").click()
    
    # Handle errors logging in
    WebDriverWait(driver=driver, timeout=10).until(
    lambda x: x.execute_script("return document.readyState === 'complete'"))
    error_message = "Incorrect username or password."
    errors = driver.find_elements_by_class_name("flash-error")
    
    # Print errors optionally
    if any(error_message in e.text for e in errors):
        print("[!] Login failed!")
    else:
        print("[+] Login successful!")


### webvitals.py

In [ ]:
# Example query for testing
test = ['47510', '47455', '40459']

In [349]:
import selenium

from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options

# Install Geckodriver
driver = webdriver.Firefox(executable_path=GeckoDriverManager(version="v0.20.0").install())

# # Enter options
# options = FirefoxOptions()
# options.add_argument("--headless")
# driver = webdriver.Firefox(options=options)

# Go to login page
driver.get("https://davos.primate.ucdavis.edu/login/")

# Login
login(username=configs.get("USERNAME")[0],
      password=configs.get("PASSWORD")[0])

# Enter Webvitals
driver.find_element_by_xpath("/html/body/ul/li[1]/a").click()

from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import numpy
import pandas as pd

animal_location(test)

print('driver Title:',driver.title)
print('Driver name:',driver.name)
print('Driver URL:',driver.current_url)
driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\Noah Siegel\.wdm\drivers\geckodriver\win64\v0.20.0\geckodriver.exe] found in cache


[+] Login successful!
      MMU     Date In   Location    Date Out        Dead  Months
1   47510  11/19/2018   NW1409-8  12/18/2018  12/18/2018    0.95
2   47510  07/16/2018   NW1411-6  11/19/2018  12/18/2018    4.10
3   47510  06/14/2018   NW1413-6  07/16/2018  12/18/2018    1.07
5   47455  10/31/2018  NW1409-32  11/30/2018  11/30/2018    1.00
6   47455  05/27/2018   SW1604-3  10/31/2018  11/30/2018    5.13
8   40459  03/20/2012   HO6107-3  04/24/2012  04/24/2012    1.13
9   40459  02/22/2012       NC17  03/20/2012  04/24/2012    0.89
10  40459  01/07/2012   HO6107-2  02/22/2012  04/24/2012    1.49
11  40459  11/23/2011       NC17  01/07/2012  04/24/2012    1.49
12  40459  11/05/2011  HO6306-53  11/23/2011  04/24/2012    0.59
13  40459  10/26/2009       NC17  11/05/2011  04/24/2012   24.33
14  40459  10/20/2009  HO6306-55  10/26/2009  04/24/2012    0.20
15  40459  08/08/2009       NC17  10/20/2009  04/24/2012    2.39
driver Title: CNPRC Web VITALS ver. 2.5.3
Driver name: firefox
Drive

### animal_info.py

In [348]:
# Relocation
def animal_location(ids):
    
    data = pd.DataFrame()
    
    for i in ids:
        
        # Go to relocation history in Webvitals
        driver.find_element_by_name("query_input").send_keys(i)
        driver.find_element_by_name("submit").click()
        driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[5]/a").click()
        
        # Extract html table
        tableelement=WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,"/html/body/table[2]/tbody/tr/td[1]/center[1]/table"))).get_attribute('outerHTML')
        table = pd.read_html(str(tableelement))[0]
        
        # Add column to specify MMU number
        table['MMU'] = i
        first_column = table.pop('MMU')
        table.insert(0, 'MMU', first_column)
        
        # Append dataframes into one dataframe
        data = data.append(table, ignore_index=True)
        
    # Clean output table   
    data = data.rename(columns={"Time at Locationyr : mon : day" : "Time"})
    data = data.join(data['Time'].str.split(':', 3, expand=True))
    data = data.rename(columns={0:"Year",1:"Month",2:"Day"})
    # Subset date of death if animal is dead
    death = (
        data
        .loc[data['Location'] == 'DEAD']
        .rename(columns={"Date In":"Dead"})
    )[['MMU', 'Dead']]
    # Merge dataframes
    df = (
        data
        .merge(death, on='MMU', how='left')
        .dropna()
    )
    # Convert age to numeric
    df['Year'] = pd.to_numeric(df['Year'])
    df['Month'] = pd.to_numeric(df['Month'])
    df['Day'] = pd.to_numeric(df['Day'])
    # List age in months
    day_year = 365.2425
    day_month = 30.436875
    df['Days'] = df.Year.mul(day_year) + df.Month.mul(day_month) + df.Day
    df['Months'] = df.Days.div(day_month)
    df.drop(['Year', 'Month', 'Day', 'Days', 'Time'],inplace=True, axis=1)
    df = df.round(2)
    # Add object to namespace
    globals()['df'] = df
    print(df)

        

### Options

In [ ]:
# Relocation
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[5]/a").click()
# Diarrhea
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[2]/tbody/tr/td[6]/a").click()
# Snomed
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[8]/a").click()
# Weight TB
driver.find_element_by_xpath("/html/body/table[1]/tbody/tr[3]/td/center/table[3]/tbody/tr/td[10]/a").click()